# Explore differences in EAP from different models

This notebook requires to have run the `0-test-model.ipynb` notebook on the different models and to have generated the different EAPs.

In [ ]:
import numpy as np
import MEAutility as mu
import matplotlib.pyplot as plt
from pathlib import Path
from bluepyopt import ephys
import neuroplotlib as nplt

import multimodalfitting as mf

%matplotlib notebook

In [ ]:
probe_type = "planar" #'linear' #
model_names = ["hay", "hay_ais", "hay_ais_hillock"] # "hay_ais" #"hallermann" #'hay'
cell_models_folder = Path("..") / "cell_models"
model_folder = cell_models_folder / f"{model_names[0]}_model"
eap_folder = Path(".")  / "eap"

In [ ]:
# load eaps
eaps = {}
for eap_file in eap_folder.iterdir():
    if "eap" in eap_file.name:
        model_name = ("_").join(eap_file.stem.split("_")[1:])
        if model_name in model_names:
            eap = np.load(eap_file)
            eaps[model_name] = eap
if len(eaps) == 0:
    raise FileNotFoundError("Could not find any EAPs to plot!")

In [ ]:
vscale = 1.5 * np.max(np.abs(list(eaps.values())))

In [ ]:
probe = mf.define_electrode(probe_type=probe_type)

for eap in eaps.values():
    assert len(probe.positions) == len(eap)

In [ ]:
fig, ax = plt.subplots(figsize=(7, 10))
norm = True
for i, (model_name, eap) in enumerate(eaps.items()):
    if norm:
        vscale = 1.5
        eap_to_plot = eap / np.max(np.abs(eap), 1, keepdims=True)
    else:
        eap_to_plot = eap
    ax = mu.plot_mea_recording(eap_to_plot, probe, colors=f"C{i}", ax=ax)
    line = ax.get_lines()[-1]
    line.set_label(model_name)
    max_channel = np.unravel_index(np.argmax(np.abs(eap)), eap.shape)[0]
    ax.plot(probe.positions[max_channel, 0], probe.positions[max_channel, 1], color=f"C{i}", marker="o")
ax.legend(loc="upper left")

In [ ]:
cell = mf.create_ground_truth_model(model_name=model_names[0], cell_model_folder=model_folder,
                                    release=True)
sim = ephys.simulators.LFPySimulator(cell, cvode_active=True, electrode=probe, mechs_folders=model_folder)

In [ ]:
mf.plot_cell(cell, sim, ax=ax)